In [ ]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.0 MB/s eta 0:

In [ ]:
import nltk
import numpy as np
import pickle as pkl

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def load_object(filename):

    file = open(filename, "rb")
    obj = pkl.load(file)
    file.close()
    return obj

initial = load_object("initial3.pkl")
emission = load_object("emission3.pkl")
transition = load_object("transition3.pkl")

word2id = load_object("word2id.pkl")
tag2id = load_object("tag2id.pkl")
id2tag = load_object("id2tag.pkl")

In [ ]:
def viterbi(text):

    sent = nltk.word_tokenize(text)


    viterbi = np.zeros((len(tag2id), len(sent)))
    backpointer = np.zeros((len(tag2id), len(sent)))

    for s in range(len(tag2id)):
        if(sent[0] in word2id):
            viterbi[s][0] = initial[s] + emission[s][word2id[sent[0]]]
        else:
            viterbi[s][0] = initial[s] + np.log(1/1000)
        backpointer[s][0] = -1

    for t in range(1, len(sent)):
        for s in range(len(tag2id)):
            temp = np.zeros((len(tag2id),))
            for s_ in range(len(tag2id)):
                if(sent[t] in word2id):
                    temp[s_] = viterbi[s_][t-1] + transition[s_][s] + emission[s][word2id[sent[t]]]
                else:
                    temp[s_] = viterbi[s_][t-1] + transition[s_][s] + np.log(1/1000)

            viterbi[s][t] = np.max(temp)
            backpointer[s][t] = np.argmax(temp)

    # bestpathprob = np.max(viterbi[:, -1])
    bestpathpointer = np.argmax(viterbi[:, -1])

    predicted_pos_sent = []

    for i in reversed(range(len(sent))):
        predicted_pos_sent.insert(0, bestpathpointer)
        bestpathpointer = int(backpointer[bestpathpointer][i])

    id_to_pos = [id2tag[i] for i in predicted_pos_sent]

    tagged_sent = [i+"_"+j for (i,j) in zip(sent, id_to_pos)]

    return(" ".join(tagged_sent))

In [ ]:
viterbi("My name is Saprativa.")

'My_DET name_NOUN is_VERB Saprativa_NOUN ._.'

In [ ]:
import gradio

gradio.Interface(viterbi, "text", "text").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://24cc8962d11dd5dde9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
